In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
# import psutil
import time

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
train_data_path = "train"
validation_data_path = "valid"
test_data_path = "test"

In [3]:
img_width, img_height = 150, 150
# channel = (3, )
input_shape = (img_width, img_height)
# input_shape = tf.expand_dims(input_shape , -1)
batch_size = 32

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)

validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  )

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_data_path,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 260915 images belonging to 14 classes.
Found 47531 images belonging to 14 classes.
Found 130645 images belonging to 13 classes.


In [6]:
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    print(num_class)
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None, input_shape[0], input_shape[1]],
                       [None, num_class])
    )
    return tf_generator


train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(validation_generator, input_shape)

14
13
14


In [7]:
from tensorflow.keras.applications import MobileNetV2

In [8]:
def create_model(img_width, img_height, num_classes):
    # base_model = VGG16(include_top=False, weights="imagenet")

    base_model = MobileNetV2(
        input_shape=input_shape + (3,),
        include_top=False,
        weights='imagenet',
        classes=num_classes,
    )

    input_layer = Input(shape=(img_width, img_height, 3))

    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Combine the input layer and the pre-trained model
    x = base_model(input_layer)
    x = GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1024, activation="relu")(x)

    for layer in base_model.layers[-5:]:
        layer.trainable = True

    # Add the output layer
    output_layer = Dense(units=num_classes, activation="softmax")(x)

    # Create and return the model
    model = Model(inputs=input_layer, outputs=output_layer)
    return model


model = create_model(img_width, img_height, 14)

In [9]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy", "categorical_crossentropy"],
)

In [10]:
model_viz = tf.keras.utils.plot_model(model,
                                      to_file='model.png',
                                      show_shapes=True,
                                      show_layer_names=True,
                                      rankdir='TB',
                                      expand_nested=True,
                                      dpi=55)
model_viz

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator) // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(
        validation_generator) // validation_generator.batch_size,
    callbacks=[early_stopping],
)

Epoch 1/20
254/254 [==============================] - 151s 580ms/step - loss: 2.4053 - accuracy: 0.2179 - categorical_crossentropy: 2.4053 - val_loss: 6.4943 - val_accuracy: 0.1834 - val_categorical_crossentropy: 6.4943
Epoch 2/20
254/254 [==============================] - 137s 540ms/step - loss: 2.2865 - accuracy: 0.2536 - categorical_crossentropy: 2.2865 - val_loss: 3.2452 - val_accuracy: 0.2140 - val_categorical_crossentropy: 3.2452
Epoch 3/20
254/254 [==============================] - 138s 544ms/step - loss: 2.2116 - accuracy: 0.2801 - categorical_crossentropy: 2.2116 - val_loss: 3.0100 - val_accuracy: 0.2758 - val_categorical_crossentropy: 3.0100
Epoch 4/20
254/254 [==============================] - 137s 540ms/step - loss: 2.1892 - accuracy: 0.2912 - categorical_crossentropy: 2.1892 - val_loss: 3.5515 - val_accuracy: 0.2174 - val_categorical_crossentropy: 3.5515
Epoch 5/20
254/254 [==============================] - 136s 534ms/step - loss: 2.1676 - accuracy: 0.2952 - categorical_cr